In [2]:
// general includes
#include <iostream>   // std::cout|endl 
#include <vector>   // std::vector
using namespace std;

## Quick Recap

- Types and value categories
    ```c++
    void init(Widget value);      // creates a copy
    void init(const Widget* ptr); // pointer is copied
    void init(const Widget& ref); // binds to rvalue or lvalue        
    void init(Widget&& ptr);      // move semantics (rvalue-reference)
    ```
- Binding rules (for references): 
    - cannot bind if constness is lost
    - cannot bind rvalue to non-const reference
    - const reference can bind to rvalues, too
    - overload resolution: distinguishes between rvalue and lvalue categories

## Goals for today
- Understand the concept of constructors/destructors 
- Understand when and why constructors/destructors have to be explicitly crafted by the programmer

# Classes: Life-cycle Hooks (Special Member Functions)

For each user-defined type (a class) the exact behavior in a fixed set of semantic embeddings is always defined 
- either by the programmer providing explicit declarations/definitions, or
- implicitly by the compiler (according to the rules of the language specification).

It is the programmers task to decide if a type requires to deviate from the default behavior provided by the compiler in one or more of these semantic embeddings.

# Construction and Destruction

We will first only look at two important semantic embeddings defining the behavior

- at the beginning of lifetime of an object (construction), and
- at the end of the lifetime of an object (destruction).

Inspect the following snippets and identify the construction and destruction events of objects of type `Widget`.

In [2]:
struct Widget {
  int a;
  double b;
  std::vector<double> c;
};

In [3]:
void func(const Widget &w3) {
  // Widget object at function scope
}

In [4]:
  Widget w1; // Widget object at scope of main function

  { // Widget object in local scope
    Widget w2;
    func(w2);
  }

**How can the programmer influence the default construction and destruction events in the above example?**
- Provide a user-defined default constructor
- Provide a user-defined destructor

**Which code was actually executed before we added user-defined constructor and destructor?**
- the versions of the *default constructor* and *destructor* implicitly generated by the compiler according to the language specification were used.


## Default constructor

A *default constructor* [(cppref)](https://en.cppreference.com/w/cpp/language/default_constructor) constructs a class object without any parameters. An example with a user-declared constructor looks like this: 


In [1]:
struct Widget {
   int a;
   double b;
   std::vector<double> c;
   Widget(); // user-declared default constructor
};


### Implicit default constructor

The implicit default constructor initializes all members using the respective *default initialization* mechanisms in order of declaration and its declaration.

Note: *default initialization* [(cppref)](https://en.cppreference.com/w/cpp/language/default_initialization) means calling the default constructor for classes and "doing nothing" for non-array built-in types.


In [6]:
struct Widget {
   int a;
   double b;
   std::vector<double> c;
   Widget() {} // default initialization in order of declaration
   //also
   //Widget()=default;
};

**What are the requirements w.r.t. the "class anatomy" to enable an implicit default constructor?**
- All members have an accessible default construction mechanism (const members and reference members do not).


## Constructors with parameters

It is common to have constructors with arguments (such constructors are never implicitly available):


In [7]:
struct Widget {
   int a;
   double b;
   std::vector<double> c;
   Widget(int a, double b, const std::vector<double> &c); // user-declared constructor with args
};

When defining such a constructor, the recommended way to initialize the members is to use the *member initializer list* [(cppref)](https://en.cppreference.com/w/cpp/language/constructor) which can look like this for the above example:

### Recommended way to init

In [8]:
struct Widget {
   int a;
   double b;
   std::vector<double> c;
   Widget(int a, double b, const std::vector<double> &c) : a(a), b(b), c(c) { }
};

When not using the *member initializer list* the constructor would look like this (not recommended):

In [9]:
struct Widget {
   int a;
   double b;
   std::vector<double> c;
   Widget(int a, double b, const std::vector<double> &c) { 
       this->a = a; // (1)
       this->b = b; // (2)
       this->c = c; // (3)
   }
};

**What are the consequences of the two different implementations above?**
- w/o initializer list more work is done: construct + copy assign (for std::vector)
- requires copy assignment to be available for all members

## Destructor

The *destructor* [(cppref)](https://en.cppreference.com/w/cpp/language/destructor) is called at the end of the lifetime of a class.
An example with a user-declared destructor looks like this: 


In [10]:
struct Widget {
   int a;
   double b;
   std::vector<double> c;
   ~Widget(); // user-declared destructor
};

The destruction mechanism then first executes the body of the destructor followed by a destruction of the members in reverse order of declaration. 


### Implicit default destructor

The implicitly-defined destructor is simply an empty body.


In [6]:
struct Widget {
   int a;
   double b;
   std::vector<double> c;
   ~Widget() {std::cout << "d";} 
 };

{
  Widget w;
}

d

**What are the requirements w.r.t. the "class anatomy" to enable an implicit default destructor?**
- no requirements, always available.

# Copying by construction or by assignment

We now look at two further important semantic embeddings defining the behavior

- when constructing an object from an already existing other object **of the same type** (copy construction), and
- when an object is assigned to another **already existing** object **of the same type** (copy assignment).

Inspect the followings snippet and identify all **copy construction** and **copy assignment** events of objects of type `Widget`.


In [12]:
struct Widget {
  int a;
  double b;
  std::vector<double> c;
};

In [13]:
void func(Widget w3) {
  // Widget object at function scope
}


In [14]:
Widget w1; // Widget object at scope of main function
{ // Widget object in local scope
  Widget w2 = w1;
  func(w2);
  w1 = w2;    
}

## Copy constructor

A *copy constructor* [(cppref)](https://en.cppreference.com/w/cpp/language/copy_constructor) constructs a class from a given lvalue reference of the same class.
An example with a user-declared copy constructor looks like this: 


In [15]:
struct Widget {
   int a;
   double b;
   std::vector<double> c;
   Widget(const Widget &other); // user-declared copy constructor
};

### Implicit copy constructor

The implicit copy constructor performs a member-wise copy in the order of declaration using *direct initialization* [(cppref)](https://en.cppreference.com/w/cpp/language/direct_initialization).

In [16]:
 struct Widget {
    int a;
    double b;
    std::vector<double> c;
    Widget(const Widget &other) /*member-wise copy*/ {} 
  };

**What are the requirements w.r.t. the "class anatomy" to enable an implicit copy constructor?**
- All class members do support copy construction.

## Copy assignment operator

A *copy assignment operator* [(cppref)](https://en.cppreference.com/w/cpp/language/copy_assignment) overloads the binary operator `=` for a class for a lvalue of the same type as right-hand-side and returns a reference.

An example with a user-declared copy assignment operator looks like this: 

In [17]:
struct Widget {
   int a;
   double b;
   std::vector<double> c;
   Widget& operator=(const Widget &rhs);  // user-declared copy assignment operator
 };

### Implicit copy assignment operator

The implicit copy assignment operator performs a member-wise assignment in the order of declaration using the copy assignment operator of the respective member (or built-in assignment for fundamental types).

In [18]:
struct Widget {
   int a;
   double b;
   std::vector<double> c;
   Widget& operator=(const Widget &rhs) { /* member-wise assignment */ return *this; }
 };


**What are the requirements w.r.t. the "class anatomy" to enable an implicit copy constructor?**
- All class members do support copy assignment.
- No reference type members present (assignment would modify referenced object). 

## Intermediate Summary

If the anatomy of the class (i.e. types of the class members) permits
the following two special member functions are implicitly available:


In [19]:
 struct Widget {
    int a;
    double b;
    std::vector<double> c;

    Widget() {} // default initialization in order of declaration
    ~Widget() {} 
    Widget(const Widget &other) /* member-wise copy */ {}     
    Widget& operator=(const Widget &rhs)  { /* member-wise assignment */ return *this; }      
  };

# Moving by construction or by assignment

The final two semantic embeddings define the behavior

- when constructing an object as a copy of an existing other object **of the same type**  which **can be moved from** (move construction), and
- when an object which **can be moved from** is assigned to another already existing object **of the same type** (move assignment).

Inspect the following snippets and identify all **copy construction** and **copy assign** events of objects of type `Widget`.


In [20]:
struct Widget {
  int a;
  double b;
  std::vector<double> c;
};

In [21]:
void func(Widget w3) {
  // Widget object at function scope
}

In [22]:
Widget w1; // Widget object at scope of main function
{ // Widget object in local scope
  Widget w2 = std::move(w1);
  func(std::move(w2));
  w1 = std::move(w2);
}

## Move constructor

A *move constructor* [(cppref)](https://en.cppreference.com/w/cpp/language/move_constructor) constructs a class from a given rvalue reference of the same class.
An example with a user-declared move constructor looks like this: 

In [23]:
 struct Widget {
    int a;
    double b;
    std::vector<double> c;
    Widget(Widget &&other);  // user-declared move constructor
  };

### Implicit move constructor

The implicit move constructor performs a member-wise move (using xvalues) in the order of declaration.

In [24]:
 struct Widget {
    int a;
    double b;
    std::vector<double> c;
    Widget(Widget &&other) /*member-wise move from xvalues*/ {} 
  };

**What are the requirements w.r.t. the "class anatomy" to enable an implicit copy constructor?**
- All class members do support move construction.

## Move assignment operator

A *Move assignment operator* [(cppref)](https://en.cppreference.com/w/cpp/language/move_assignment) overloads the binary operator `=` for a class for a rvalue reference of the same type as right-hand-side and returns a reference.
An example with a user-declared move assignment operator looks like this: 


In [25]:
 struct Widget {
    int a;
    double b;
    std::vector<double> c;
    Widget& operator=(Widget &&rhs); // user-declared move assignment operator
  };

### Implicit move assignment operator

The implicit move assignment operator performs a member-wise assignment in the order of declaration using the move assignment operator of the respective member (or built-in assignment for fundamental types).

In [26]:
 struct Widget {
    int a;
    double b;
    std::vector<double> c;
    Widget& operator=(Widget &&rhs) { /* member-wise move assignment */ return *this; }
  };

**What are the requirements w.r.t. the "class anatomy" to enable an implicit copy constructor?**
- All class members do support move assignment.
- No reference type members present (assignment would modify referenced object). 

## Summary: the full set of special member functions (1+5)

If the anatomy of the class (i.e. types of the class members) permits
the following two special member functions are implicitly available:


In [27]:
  struct Widget {
    int a;
    double b;
    std::vector<double> c;

    Widget() {}  // default initialization in order of declaration
    ~Widget() {}  
    Widget(const Widget &other)  /* member-wise copy */ {}     
    Widget& operator=(const Widget &rhs) { /* member-wise copy assignment */ return *this; }      
    Widget(Widget &&other)  /*member-wise move from xvalues*/ {} 
    Widget& operator=(Widget &&rhs)  { /* member-wise move assignment */ return *this; }
  };


# Guidelines

The rules under which conditions exactly each of the 6 special member functions listed above is *implicitly-defined* (i.e. functionally present) for a class are quite complex and make it easy to get lost. This is why "guideline rules" [(cppref)](https://en.cppreference.com/w/cpp/language/rule_of_three) are very prominent for this very important part of the C++ language specification. 

## "Rule of Three": the resource-owning class

If a class owns a resource (by means of a raw pointer, or file handle), this definitely requires user-defined versions for
- the destructor (to release the resource),
- the copy constructor (to avoid a "sallow copy"),
- the copy assignment operator (to avoid leaking the resource held before the assignment).

Below is a simple example of a class owning a resource and providing user-defined versions for the above mentioned special member functions (and a constructor).

In [28]:
struct Resource {
  double *data; // pointer to array in dynamic memory
  size_t size;  // length of array

  Resource(size_t size) : size(size), data(new double[size]) {}

  ~Resource() { delete[] data; }
  Resource(const Resource &other)
      : size(other.size), data(new double[other.size]) {
    std::copy(other.data, other.data + size, data);
  }
  Resource &operator=(const Resource &rhs) {
    Resource tmp(rhs);
    std::swap(tmp.size, size);
    std::swap(tmp.data, data);
    return *this;
  }
};

**Why is it impossible to rely on the compiler generated versions of the special member functions, are they even available for this class?**
- there is no way for the compiler to infer if a raw pointer is actually a dynamic resource.

**Can the above class be moved?**
- no, move-related special member functions are not declared implicitly if other user-defined special member functions are present.

## "Rule of Five": the resource owning class + move semantics

This rules says that a resource owning class should provide all 5 special member functions:
- the 3 from "rule of three" and
- the 2 move-semantics related functions

If we extend the `Resource` class from above this looks like this:

In [29]:
struct Resource {

  double *data; // pointer to array in dynamic memory
  size_t size;  // length of array

  Resource(Resource &&other) : size(0), data(nullptr) {
    std::swap(other.size, size);
    std::swap(other.data, data);
  }
  Resource &operator=(Resource &&rhs) {
    std::swap(rhs.size, size);
    std::swap(rhs.data, data);
    return *this;
  }
  
};

## "Rule of Zero": all non-owning classes should rely on all 5 compiler generated special member functions

Assuming resource owning classes exclusively deal with the ownership by means of implementing all required special member functions, all other classes (i.e. classes not owning resources directly) should not required user-defined implementations for any of the 5 special member functions.

Using a resource-owning class (here we use `std::vector`) in a composed class looks like this:


In [30]:
using Resource = std::vector<double>;

struct Composed {
  int id;
  std::string name;
  Resource res; 
  Composed(int id, const std::string &name, const Resource &res)
      : id(id), name(name), res(res) {}
};

As the `std::vector` class implements all special member functions, also the `Composed` class can be used in all expected semantic embeddings:

In [31]:
 Composed a(1, "name1", Resource(10));
 Composed b(2, "name2", Resource(20));
 Composed c(a);
 c = b;
 Composed d(std::move(b));
 d = std::move(c);

**What if `Resource` would not have implemented the move-related special member functions, would the snippet above still compile?**
- yes, if move semantics are not available for a member, the overload resolution resorts to selecting the respective special member functions for copy-semantics (const lvalue references can also bind to rvalues).


# Summary

- "RAII": wrap raw resources in a class.
- "Rule of Three/Five": The *lifecycle-hooks* (ctors/assignments/dtor) allow to hide all required actions in all semantic embeddings.
- "Rule of Zero": The default implementations (member-wise copy/move/assign/move-assign) allow to design composed classes without the need for user-defined special member functions.